In [8]:
from langgraph.graph import StateGraph, START, END

In [2]:
from typing import TypedDict


In [3]:
class State(TypedDict):
    topic: str
    joke: str

In [31]:
def refin_topic(state: State):
    return {"topic": state["topic"] + " and cats"}

In [32]:
def generate_joke(state: State):
    return {"joke": f"This is a joke about {state['topic']}"}

In [33]:
graph = (
    StateGraph(State).add_node(refin_topic).add_node(generate_joke).add_edge(START, "refin_topic").add_edge("refin_topic", "generate_joke")
).compile()

In [34]:
for chunk in graph.stream(
    {"topic": "ice cream"},
    stream_mode = "values"
):
    print(chunk)

{'topic': 'ice cream'}
{'topic': 'ice cream and cats'}
{'topic': 'ice cream and cats', 'joke': 'This is a joke about ice cream and cats'}


In [35]:
for chunk in graph.stream(
    {"topic": "ice cream"},
    stream_mode = "updates"
):
    print(chunk)

{'refin_topic': {'topic': 'ice cream and cats'}}
{'generate_joke': {'joke': 'This is a joke about ice cream and cats'}}


In [36]:
for chunk in graph.stream(
    {"topic": "ice cream"},
    stream_mode = "debug"
):
    print(chunk)

{'type': 'task', 'timestamp': '2025-02-24T06:11:39.245882+00:00', 'step': 1, 'payload': {'id': 'b9845c79-01d7-bdc5-f08f-99924c658e44', 'name': 'refin_topic', 'input': {'topic': 'ice cream'}, 'triggers': ['start:refin_topic']}}
{'type': 'task_result', 'timestamp': '2025-02-24T06:11:39.246389+00:00', 'step': 1, 'payload': {'id': 'b9845c79-01d7-bdc5-f08f-99924c658e44', 'name': 'refin_topic', 'error': None, 'result': [('topic', 'ice cream and cats')], 'interrupts': []}}
{'type': 'task', 'timestamp': '2025-02-24T06:11:39.246718+00:00', 'step': 2, 'payload': {'id': 'c0ab5231-5023-6056-36bb-60834ea47245', 'name': 'generate_joke', 'input': {'topic': 'ice cream and cats'}, 'triggers': ['refin_topic']}}
{'type': 'task_result', 'timestamp': '2025-02-24T06:11:39.247014+00:00', 'step': 2, 'payload': {'id': 'c0ab5231-5023-6056-36bb-60834ea47245', 'name': 'generate_joke', 'error': None, 'result': [('joke', 'This is a joke about ice cream and cats')], 'interrupts': []}}


In [37]:
for chunk in graph.stream(
    {"topic": "ice cream"},
    stream_mode = ["debug","values","updates"]
):
    print(chunk)

('values', {'topic': 'ice cream'})
('debug', {'type': 'task', 'timestamp': '2025-02-24T06:12:07.288044+00:00', 'step': 1, 'payload': {'id': '8af0535c-ffd7-18fe-bc4e-eaeccf224582', 'name': 'refin_topic', 'input': {'topic': 'ice cream'}, 'triggers': ['start:refin_topic']}})
('updates', {'refin_topic': {'topic': 'ice cream and cats'}})
('debug', {'type': 'task_result', 'timestamp': '2025-02-24T06:12:07.288904+00:00', 'step': 1, 'payload': {'id': '8af0535c-ffd7-18fe-bc4e-eaeccf224582', 'name': 'refin_topic', 'error': None, 'result': [('topic', 'ice cream and cats')], 'interrupts': []}})
('values', {'topic': 'ice cream and cats'})
('debug', {'type': 'task', 'timestamp': '2025-02-24T06:12:07.289258+00:00', 'step': 2, 'payload': {'id': '874c8e97-7183-c3b6-3301-898ab671f34f', 'name': 'generate_joke', 'input': {'topic': 'ice cream and cats'}, 'triggers': ['refin_topic']}})
('updates', {'generate_joke': {'joke': 'This is a joke about ice cream and cats'}})
('debug', {'type': 'task_result', 'time

In [17]:
from langchain_openai import ChatOpenAI

In [18]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [20]:
def generate_joke(state: State):
    llm_response = llm.invoke(
        [
            {
                "role": "user", "content": f"Generate a joke about {state['topic']}"
            }
        ]
    )
    return {"joke": llm_response.content}

In [21]:
graph = StateGraph(State)
graph.add_node(refin_topic)
graph.add_node(generate_joke)
graph.add_edge(START, "refin_topic")
graph.add_edge('refin_topic', "generate_joke")
graph.add_edge("generate_joke", END)

In [22]:
graph = graph.compile()

In [24]:
for message_chunk, metadata  in graph.stream(
    {"topic": "ice cream"},
    stream_mode="messages"
):
    if message_chunk.content:
        print(message_chunk.content, end="|", flush=True)

Why| did| the| cat| sit| on| the| ice| cream| cone|?

|Because| it| wanted| to| be| a| little| "|p|urr|-f|ect|"| scoop|!| 🍦|😸|

In [25]:
from langgraph.types import StreamWriter

In [27]:
def generate_joke(state: State, writer: StreamWriter):
    writer({"custom key": "This  a custom stream data"})
    return {"joke": f"This is a joke about {state['topic']}"}

In [28]:
graph = StateGraph(State)
graph.add_node(refin_topic)
graph.add_node(generate_joke)
graph.add_edge(START, "refin_topic")
graph.add_edge('refin_topic', "generate_joke")
graph.add_edge("generate_joke", END)

In [29]:
graph = graph.compile()

In [30]:
for chunk in graph.stream({
    "topic": "ice cream"
},
stream_mode="custom"
):
    print(chunk)

{'custom key': 'This  a custom stream data'}
